Election de 2022 premier tour

In [2]:
import numpy as np
import pandas as pd
import os
os.listdir()

['immigration.ipynb',
 'migrations',
 'election2017.ipynb',
 '.DS_Store',
 'alembic.ini',
 'economie',
 'enums',
 'depense.ipynb',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'population.ipynb',
 'scripts',
 'db',
 'securite.ipynb',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'data',
 'chomage.ipynb']

#### Importer la database

Import des datasets.

In [3]:
df = pd.read_excel('./data/economie/menage/depenses.xlsx')

Visualise toutes les colones

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Année,Dépenses pré-engagées,Dépenses liées au logement,Services de télévision et de télécommunications
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5
...,...,...,...,...
58,2018,29.1,22.7,2.1
59,2019,28.8,22.6,2.1
60,2020,28.5,22.5,2.1
61,2021,28.5,22.5,2.1


Renommer les colonnes 

In [9]:
df.columns = [
    "annee", "pre_engagees", "logement", "service_multimedia",
]
df.head()

,annee,pre_engagees,logement,service_multimedia
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5


Normalisation structurelle

In [ ]:
# Conversion de l'année en int

df["annee"] = (
    df["annee"]
    .astype(str)
    .str.extract(r"(\d{4})")[0]  # sécurité si texte parasite
    .astype(int)
)


# Conversion des pourcentages en float

colonnes_pourcentage = [
    "pre_engagees", "logement", "service_multimedia",
]

for col in colonnes_pourcentage:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace("%", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )


In [16]:
df.head()

,annee,pre_engagees,logement,service_multimedia
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   annee               63 non-null     int64  
 1   pre_engagees        63 non-null     float64
 2   logement            63 non-null     float64
 3   service_multimedia  63 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 2.1 KB


In [20]:
df.shape

(63, 4)

In [11]:
from db import SessionLocal
from models import Immigration

session = SessionLocal()

# Insertion des données
for row in df.itertuples(index=False):
    record = Immigration(annee=row.annee, pct_immigration=row.pct_immigration)
    session.add(record)

session.commit()
print("Insertion terminée pour les années 2007 à 2024.")


2026-01-08 05:10:38,380 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2026-01-08 05:10:38,381 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-08 05:10:38,392 INFO sqlalchemy.engine.Engine select current_schema()
2026-01-08 05:10:38,392 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-08 05:10:38,396 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2026-01-08 05:10:38,398 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-08 05:10:38,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-08 05:10:38,409 INFO sqlalchemy.engine.Engine INSERT INTO immigration (annee, pct_immigration) SELECT p0::INTEGER, p1::FLOAT FROM (VALUES (%(annee__0)s, %(pct_immigration__0)s, 0), (%(annee__1)s, %(pct_immigration__1)s, 1), (%(annee__2)s, %(pct_immigration__2)s, 2), (%(annee__3)s, %(pct_immigrat ... 1240 characters truncated ...  imp_sen(p0, p1, sen_counter) ORDER BY sen_counter RETURNING immigration.id, immigration.id AS id__1
2026-01-08 05:10:38,411 INFO sqlalchemy.e

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "immigration" does not exist
LINE 1: INSERT INTO immigration (annee, pct_immigration) SELECT p0::...
                    ^

[SQL: INSERT INTO immigration (annee, pct_immigration) SELECT p0::INTEGER, p1::FLOAT FROM (VALUES (%(annee__0)s, %(pct_immigration__0)s, 0), (%(annee__1)s, %(pct_immigration__1)s, 1), (%(annee__2)s, %(pct_immigration__2)s, 2), (%(annee__3)s, %(pct_immigrat ... 1240 characters truncated ...  imp_sen(p0, p1, sen_counter) ORDER BY sen_counter RETURNING immigration.id, immigration.id AS id__1]
[parameters: {'annee__0': 2024, 'pct_immigration__0': 11.3, 'annee__1': 2023, 'pct_immigration__1': 10.7, 'annee__2': 2022, 'pct_immigration__2': 10.32776865187, 'annee__3': 2021, 'pct_immigration__3': 10.2, 'annee__4': 2020, 'pct_immigration__4': 10.2, 'annee__5': 2019, 'pct_immigration__5': 10.0, 'annee__6': 2018, 'pct_immigration__6': 9.8, 'annee__7': 2017, 'pct_immigration__7': 9.7, 'annee__8': 2016, 'pct_immigratio